In [1]:
"""
    製作一個客製menu

"""


#做一個menu的基本框架

menuRawData="""
{
  "size": {
    "width": 2500,
    "height": 843
  },
  "selected": true,
  "name": "自定義菜單-2",
  "chatBarText": "功能選單",
  "areas": [
    {
      "bounds": {
        "x": 66,
        "y": 155,
        "width": 510,
        "height": 510
      },
      "action": {
        "type": "postback",
        "data": "type=question&question_type=sa"
      }
    },
    {
      "bounds": {
        "x": 676,
        "y": 155,
        "width": 510,
        "height": 510
      },
      "action": {
        "type": "postback",
        "data": "type=question&question_type=develop"
      }
    },
    {
      "bounds": {
        "x": 1296,
        "y": 155,
        "width": 510,
        "height": 510
      },
      "action": {
        "type": "postback",
        "data": "type=question&question_type=sysops"
      }
    },
    {
      "bounds": {
        "x": 1906,
        "y": 155,
        "width": 510,
        "height": 510
      },
      "action": {
        "type":"message",
        "text":"more"
      }
    }   
  ]
}
"""

In [2]:
"""
    user_id：就是姓名
    access_token：就像是ssh的私鑰
    secret_key：就是身份證字號，證明是line本人

"""

#擷取檔案內的變數，下面會使用

import json
secretFileContentJson=json.load(open("./secret_key",'r'))
menu_id = secretFileContentJson.get("rich_menu_id")
access_token = secretFileContentJson.get("channel_access_token")
sercret_key = secretFileContentJson.get("secret_key")
user_id = secretFileContentJson.get("self_user_id")
#印出來測試使用
# print(menu_id)
# print(access_token)
# print(sercret_key)
# print(user_id)

In [3]:
"""
    將上面做好的menu框架post給line
    line會製作一個menu_id回傳過來
    之後就都是靠menu_id來連動menu
    一個bot最多一千個menu

"""


#使用requests來傳送封包

import requests

#讀上面的menu框架
menuJson=json.loads(menuRawData)

#line的位置
createMenuEndpoint = 'https://api.line.me/v2/bot/richmenu'
#http的header靠Authorization來辨識你是誰
createMenuRequestHeader={'Content-Type':'application/json','Authorization':'Bearer %s' % access_token }
#使用post的方式新增menu進line，會回傳一個menu_id
lineCreateMenuResponse = requests.post(createMenuEndpoint,headers=createMenuRequestHeader,data=json.dumps(menuJson))

#看是否正確回覆
print(lineCreateMenuResponse)
#取出menu_id
print(lineCreateMenuResponse.text)

richMenuId_dict = json.loads(lineCreateMenuResponse.text)
richMenuId_dict['richMenuId']

#要將印出來的richMenuId的值回填到secret_key檔案中的 rich_menu_id
#存檔後要再執行上一個cell



<Response [200]>
{"richMenuId":"richmenu-e0828d91dd90f28d6d808b0034f62b69"}


'richmenu-e0828d91dd90f28d6d808b0034f62b69'

In [4]:
'''
    得到menu_id後
    再對menu進行更多的設定

'''
import requests

# 從secret_key取得菜單Id 
uploadRichMenuId=richMenuId_dict['richMenuId']
print(uploadRichMenuId)

# 設定Line的遠端位置
uploadMenuEndpoint='https://api.line.me/v2/bot/richmenu/%s/content' % uploadRichMenuId
print(uploadMenuEndpoint)

# 設定消息的基本安全憑證
uploadMenuRequestHeader={'Content-Type':'image/jpeg','Authorization':'Bearer %s' % secretFileContentJson["channel_access_token"]}

# 上傳照片，這個照片檔要特別注意檔案格式，要是2500X843跟上面製作的rich_menu格式相符
uploadImageFile=open("./images/menu_2.jpg",'rb')
# 用request傳post http給line對menu_id做設定
lineUploadMenuResponse=requests.post(uploadMenuEndpoint,headers=uploadMenuRequestHeader,data=uploadImageFile)

# 檢查是否成功
print(lineUploadMenuResponse)
print(lineUploadMenuResponse.text)

richmenu-e0828d91dd90f28d6d808b0034f62b69
https://api.line.me/v2/bot/richmenu/richmenu-e0828d91dd90f28d6d808b0034f62b69/content
<Response [200]>
{}


In [5]:
"""

    將做好的menu_id跟menu資料
    post 到 API server
    
"""

import requests
#做一個將已經做好的menu進資料庫的動作
#網址
url = "http://chatbot_api:5000/menu"
#表頭
Header={'Content-Type':'application/json'}
#傳送的資料，裝menu的格式
#menu_id 不能使用變數呼叫 
payload = {
    "menu_id": menu_id,
    "menu_content": menuRawData
}

#傳送封包
Response=requests.post(url,headers=Header,data=json.dumps(payload))
#檢驗
print(Response)
print(Response.text)



ConnectionError: HTTPConnectionPool(host='chatbot_api', port=5000): Max retries exceeded with url: /menu (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f353ebfee80>: Failed to establish a new connection: [Errno 111] Connection refused',))

In [18]:
"""
    
    查詢line@綁定的menu

"""

# 取出用戶id，設定Line的遠端位置
user_id = secretFileContentJson.get("self_user_id")
userMenuEndpoint='https://api.line.me/v2/bot/user/%s/richmenu' % (user_id)
print(userMenuEndpoint)

# 設定消息的基本安全憑證
userMenuRequestHeader={'Authorization':'Bearer %s' % secretFileContentJson["channel_access_token"]}

# 發送消息告知
lineUserMenuResponse=requests.get(userMenuEndpoint,headers=userMenuRequestHeader)
print(lineUserMenuResponse)
print(lineUserMenuResponse.text)

https://api.line.me/v2/bot/user/U7bd8f7a1c9e46ae01e0a06f6b2b8fb27/richmenu
<Response [404]>
{"message":"the user has no richmenu","details":[]}


In [10]:
"""

    刪除綁定在line@身上的menu

"""

#刪除原有榜定的menu
# 取出用戶id，設定Line的遠端位置
userMenuEndpoint='https://api.line.me/v2/bot/user/%s/richmenu' % (secretFileContentJson["self_user_id"])
print(userMenuEndpoint)

# 設定消息的基本安全憑證
userMenuRequestHeader={'Authorization':'Bearer %s' % secretFileContentJson["channel_access_token"]}

# 發送消息告知
lineUnregisterUserMenuResponse=requests.delete(userMenuEndpoint,headers=userMenuRequestHeader)
print(lineUnregisterUserMenuResponse)
print(lineUnregisterUserMenuResponse.text)


https://api.line.me/v2/bot/user/U7bd8f7a1c9e46ae01e0a06f6b2b8fb27/richmenu
<Response [200]>
{}
